<a href="https://colab.research.google.com/github/EugeniaBar/Proyectoscompletos/blob/main/PROCESAMIENTO_DEL_HABLA_COMPLETO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Sentimientos y Reducción del Corpus en Reseñas Cinematográficas en Español: Un Enfoque Práctico de PLN

**Técnicas del Procesamiento del Habla - ABP 2025**

## 1. Carga del Dataset
En esta sección se importa el dataset de reseñas de películas en español y se realiza una exploración inicial de los datos.

In [ ]:
from google.colab import files

# Subir dataset
uploaded = files.upload()

In [ ]:
import pandas as pd

df = pd.read_csv('IMDB Dataset SPANISH.csv')
df.head()

,Unnamed: 0,review_en,review_es,sentiment,sentimiento
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   50000 non-null  int64 
 1   review_en    50000 non-null  object
 2   review_es    50000 non-null  object
 3   sentiment    50000 non-null  object
 4   sentimiento  50000 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.9+ MB


In [ ]:
# Columna de sentimiento
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [ ]:
# Reseñas en español
df['review_es'].sample(5, random_state=42)

33553    Realmente me gustó este verano debido a la apa...
9427     No hay muchos programas de televisión que apel...
199      La película llega rápidamente a una escena pri...
12447    ¡Jane Austen definitivamente aprobaría este! G...
39489    Las expectativas eran un poco altas para mí cu...
Name: review_es, dtype: object

## 2. Limpieza y Normalización del Texto
Se aplican técnicas de preprocesamiento: eliminación de símbolos, conversión a minúsculas, eliminación de acentos y espacios.

### Limpieza de Texto con Expresiones Regulares (Regex)

En esta sección preprocesamos el texto original de las reseñas utilizando expresiones regulares. Este paso es fundamental para preparar el texto antes de realizar análisis más avanzados. Se realiza la limpieza en etapas, mostrando el resultado intermedio después de cada una de ellas:

1. **Eliminación de signos de puntuación y caracteres especiales:** se eliminan elementos como comas, puntos, signos de exclamación, etc., que no aportan valor semántico en este contexto.
2. **Conversión a minúsculas:** todas las letras se convierten a minúsculas para unificar la representación de las palabras.
3. **Eliminación de espacios en blanco adicionales:** se remueven espacios múltiples y se limpian los espacios al inicio y al final de cada texto.
4. **Eliminación de acentos (opcional):** se normalizan los caracteres acentuados para evitar duplicaciones de palabras equivalentes con y sin tilde.

Este proceso permite obtener una versión más limpia y uniforme de las reseñas, facilitando la tokenización y el análisis posterior.


In [ ]:
# importamos las librerías necesarias
import re
import pandas as pd
from unidecode import unidecode

In [ ]:
# eliminamos signos de puntuación y caracteres especiales
df['review_sin_puntuacion'] = df['review_es'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# muestra comparativa
print("Paso 1: Eliminación de signos de puntuación y caracteres especiales")
display(df[['review_es', 'review_sin_puntuacion']].sample(3, random_state=1))

Paso 1: Eliminación de signos de puntuación y caracteres especiales


,review_es,review_sin_puntuacion
26247,"Sin héroes muertos, se obtienen líneas estúpid...",Sin héroes muertos se obtienen líneas estúpida...
35067,Pensé que tal vez ... tal vez esto podría ser ...,Pensé que tal vez tal vez esto podría ser bue...
34590,"Un equipo militar de élite americano que, por ...",Un equipo militar de élite americano que por s...


In [ ]:
# convertimos a minúsculas
df['review_minusculas'] = df['review_sin_puntuacion'].apply(lambda x: x.lower())

# muestra comparativa
print("Paso 2: Conversión a minúsculas")
display(df[['review_sin_puntuacion', 'review_minusculas']].sample(3, random_state=2))

Paso 2: Conversión a minúsculas


,review_sin_puntuacion,review_minusculas
23656,Ron Hall saca una triple amenaza mientras escr...,ron hall saca una triple amenaza mientras escr...
27442,El primero en la serie fue brillante fácilment...,el primero en la serie fue brillante fácilment...
40162,Me encantó esta película porque Bobbie Phillip...,me encantó esta película porque bobbie phillip...


In [ ]:
# eliminamos espacios en blanco
df['review_espacios_limpios'] = df['review_minusculas'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# muestra comparativa
print("Paso 3: Eliminación de espacios en blanco")
display(df[['review_minusculas', 'review_espacios_limpios']].sample(3, random_state=3))

Paso 3: Eliminación de espacios en blanco


,review_minusculas,review_espacios_limpios
35437,chris y andre son dos adolescentes promedio or...,chris y andre son dos adolescentes promedio or...
16296,robin williams muestra sus talentos de pie y a...,robin williams muestra sus talentos de pie y a...
23122,no puedo creer que hagan este tipo de suciedad...,no puedo creer que hagan este tipo de suciedad...


In [ ]:
# eliminamos acentos
df['review_sin_acentos'] = df['review_espacios_limpios'].apply(lambda x: unidecode(x))

# muestra comparativa
print("Paso 4: Eliminación de acentos (opcional)")
display(df[['review_espacios_limpios', 'review_sin_acentos']].sample(3, random_state=4))

Paso 4: Eliminación de acentos (opcional)


,review_espacios_limpios,review_sin_acentos
16477,a primera vista del lema de la trama pensé que...,a primera vista del lema de la trama pense que...
5969,buen señor cómo terminó esto en nuestro reprod...,buen senor como termino esto en nuestro reprod...
46459,dos hechiceros luchan en la cuarta dimensión u...,dos hechiceros luchan en la cuarta dimension u...


In [ ]:
# mostramos las 10 primeras filas de todas las etapas del preprocesamiento para comprobar los cambios
df[['review_es',
    'review_sin_puntuacion',
    'review_minusculas',
    'review_espacios_limpios',
    'review_sin_acentos']].head(10)

,review_es,review_sin_puntuacion,review_minusculas,review_espacios_limpios,review_sin_acentos
0,Uno de los otros críticos ha mencionado que de...,Uno de los otros críticos ha mencionado que de...,uno de los otros críticos ha mencionado que de...,uno de los otros críticos ha mencionado que de...,uno de los otros criticos ha mencionado que de...
1,Una pequeña pequeña producción.La técnica de f...,Una pequeña pequeña producciónLa técnica de fi...,una pequeña pequeña producciónla técnica de fi...,una pequeña pequeña producciónla técnica de fi...,una pequena pequena produccionla tecnica de fi...
2,Pensé que esta era una manera maravillosa de p...,Pensé que esta era una manera maravillosa de p...,pensé que esta era una manera maravillosa de p...,pensé que esta era una manera maravillosa de p...,pense que esta era una manera maravillosa de p...
3,"Básicamente, hay una familia donde un niño peq...",Básicamente hay una familia donde un niño pequ...,básicamente hay una familia donde un niño pequ...,básicamente hay una familia donde un niño pequ...,basicamente hay una familia donde un nino pequ...
4,"El ""amor en el tiempo"" de Petter Mattei es una...",El amor en el tiempo de Petter Mattei es una p...,el amor en el tiempo de petter mattei es una p...,el amor en el tiempo de petter mattei es una p...,el amor en el tiempo de petter mattei es una p...
5,Probablemente mi película favorita de todos lo...,Probablemente mi película favorita de todos lo...,probablemente mi película favorita de todos lo...,probablemente mi película favorita de todos lo...,probablemente mi pelicula favorita de todos lo...
6,Seguro que me gustaría ver una resurrección de...,Seguro que me gustaría ver una resurrección de...,seguro que me gustaría ver una resurrección de...,seguro que me gustaría ver una resurrección de...,seguro que me gustaria ver una resurreccion de...
7,"Este espectáculo fue una idea increíble, fresc...",Este espectáculo fue una idea increíble fresca...,este espectáculo fue una idea increíble fresca...,este espectáculo fue una idea increíble fresca...,este espectaculo fue una idea increible fresca...
8,Alentados por los comentarios positivos sobre ...,Alentados por los comentarios positivos sobre ...,alentados por los comentarios positivos sobre ...,alentados por los comentarios positivos sobre ...,alentados por los comentarios positivos sobre ...
9,"Si te gusta la risa original desgarradora, te ...",Si te gusta la risa original desgarradora te g...,si te gusta la risa original desgarradora te g...,si te gusta la risa original desgarradora te g...,si te gusta la risa original desgarradora te g...


In [ ]:
# guardamos el texto de las reseñas en una lista para su uso posterior
reseñas = df['review_sin_acentos'].tolist()
# guardamos la columna con el texto limpio de las reseñas en una columna nueva
df['texto_limpio'] = df['review_sin_acentos']

### Tokenización de Texto

La tokenización es el proceso de dividir un texto en unidades más pequeñas llamadas *tokens*, que generalmente corresponden a palabras. Este paso es esencial para el análisis posterior como el conteo de palabras o la creación de modelos de representación de texto. En esta sección se aplicarán tres enfoques diferentes:

1. **Tokenización simple:** división básica del texto utilizando espacios como separadores.
2. **Tokenización con NLTK:** se usa la biblioteca NLTK que proporciona herramientas específicas para el idioma español.
3. **Tokenización con scikit-learn (CountVectorizer):** permite tokenizar y vectorizar simultáneamente, generando una representación numérica del texto.

Cada método será aplicado sobre las reseñas limpias y se compararán los resultados para observar cómo manejan signos de puntuación, palabras compuestas y caracteres especiales.


In [ ]:
# Tokenización simple
df['tokens_simple'] = df['texto_limpio'].apply(lambda x: x.split())

# mostramos algunos resultados
print("Tokenización simple (primeras 3 reseñas):")
df[['texto_limpio', 'tokens_simple']].head(25)


Tokenización simple (primeras 3 reseñas):


,texto_limpio,tokens_simple
0,uno de los otros criticos ha mencionado que de...,"[uno, de, los, otros, criticos, ha, mencionado..."
1,una pequena pequena produccionla tecnica de fi...,"[una, pequena, pequena, produccionla, tecnica,..."
2,pense que esta era una manera maravillosa de p...,"[pense, que, esta, era, una, manera, maravillo..."
3,basicamente hay una familia donde un nino pequ...,"[basicamente, hay, una, familia, donde, un, ni..."
4,el amor en el tiempo de petter mattei es una p...,"[el, amor, en, el, tiempo, de, petter, mattei,..."
5,probablemente mi pelicula favorita de todos lo...,"[probablemente, mi, pelicula, favorita, de, to..."
6,seguro que me gustaria ver una resurreccion de...,"[seguro, que, me, gustaria, ver, una, resurrec..."
7,este espectaculo fue una idea increible fresca...,"[este, espectaculo, fue, una, idea, increible,..."
8,alentados por los comentarios positivos sobre ...,"[alentados, por, los, comentarios, positivos, ..."
9,si te gusta la risa original desgarradora te g...,"[si, te, gusta, la, risa, original, desgarrado..."


In [ ]:
# Tokenización con NLTK
# instalamos e importamos NLTK
#!pip install nltk
import nltk
from nltk.tokenize import TweetTokenizer

In [ ]:
# instanciamos el tokenizador
tknzr = TweetTokenizer()

# aplicar a la columna
df['tokens_nltk'] = df['texto_limpio'].apply(lambda x: tknzr.tokenize(x))

# muestra
df[['texto_limpio', 'tokens_nltk']].head(25)

,texto_limpio,tokens_nltk
0,uno de los otros criticos ha mencionado que de...,"[uno, de, los, otros, criticos, ha, mencionado..."
1,una pequena pequena produccionla tecnica de fi...,"[una, pequena, pequena, produccionla, tecnica,..."
2,pense que esta era una manera maravillosa de p...,"[pense, que, esta, era, una, manera, maravillo..."
3,basicamente hay una familia donde un nino pequ...,"[basicamente, hay, una, familia, donde, un, ni..."
4,el amor en el tiempo de petter mattei es una p...,"[el, amor, en, el, tiempo, de, petter, mattei,..."
5,probablemente mi pelicula favorita de todos lo...,"[probablemente, mi, pelicula, favorita, de, to..."
6,seguro que me gustaria ver una resurreccion de...,"[seguro, que, me, gustaria, ver, una, resurrec..."
7,este espectaculo fue una idea increible fresca...,"[este, espectaculo, fue, una, idea, increible,..."
8,alentados por los comentarios positivos sobre ...,"[alentados, por, los, comentarios, positivos, ..."
9,si te gusta la risa original desgarradora te g...,"[si, te, gusta, la, risa, original, desgarrado..."


In [ ]:
# Tokenización con CountVectorizer (scikit-learn)
# instalamos e importamos la librería
#!pip install scikit-learn
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# instanciamos el vectorizador
vectorizador = CountVectorizer()

# aplicamos fit y transform para generar la matriz BoW
X = vectorizador.fit_transform(df['texto_limpio'])

# extraemos los tokens identificados por el vectorizador
tokens_sklearn = vectorizador.get_feature_names_out()

# Convertimos una parte  de la matriz BoW a un DataFrame para ver las palabras
df_bow_sample = pd.DataFrame(X[:5].toarray(), columns=tokens_sklearn)

# Para cada fila (reseña), mostramos solo las palabras presentes (con valor > 0)
for idx, row in df_bow_sample.iterrows():
    palabras_presentes = row[row > 0].index.tolist()
    print(f"Reseña {idx + 1}:")
    print(palabras_presentes)
    print("-" * 50)


Reseña 1:
['acostumbre', 'acuerdos', 'adentro', 'agenda', 'al', 'alejaran', 'alta', 'altos', 'apodo', 'asi', 'atractivo', 'atreverian', 'audiencias', 'bonitas', 'brutalidad', 'callejeras', 'celulas', 'centra', 'city', 'ciudad', 'clase', 'clasico', 'comodo', 'con', 'confia', 'conmigo', 'contacto', 'convencionales', 'convirtieron', 'corazon', 'cosa', 'cristianos', 'criticos', 'cuando', 'dado', 'de', 'debe', 'debido', 'debiles', 'decir', 'del', 'derecha', 'desagradable', 'desarrolle', 'despues', 'diria', 'donde', 'drogas', 'educada', 'el', 'ello', 'em', 'emeralda', 'en', 'encanto', 'encuentran', 'enfrentan', 'enganchado', 'episodio', 'es', 'escenas', 'eso', 'espectaculo', 'espectaculos', 'esposas', 'estaba', 'estado', 'estan', 'estara', 'este', 'esto', 'exactamente', 'experiencia', 'experimental', 'extrae', 'falta', 'fariarios', 'frentes', 'fue', 'gangstas', 'golpeo', 'grafica', 'guardias', 'gusto', 'ha', 'habilidades', 'hacia', 'hardcore', 'he', 'hecho', 'hogar', 'imagenes', 'incomodo', 

### Comparación de Métodos de Tokenización

A continuación, se comparan los resultados obtenidos al aplicar tres métodos distintos de tokenización sobre una misma reseña:

1. **Tokenización Simple:** divide el texto usando espacios como delimitadores. Es rápida y fácil de implementar, pero no considera puntuación ni estructuras lingüísticas.
2. **Tokenización con NLTK (`TweetTokenizer`):** separa correctamente signos de puntuación y palabras, ofreciendo una segmentación más precisa. Es adecuada para textos informales y no requiere recursos externos.
3. **Tokenización con `CountVectorizer` de scikit-learn:** tokeniza y vectoriza al mismo tiempo, generando un vocabulario basado en frecuencia. No conserva el orden de los tokens y elimina los duplicados.

Esta comparación permite observar cómo cada método maneja elementos como puntuación, tildes y signos especiales, lo cual impactará en etapas posteriores como el conteo de palabras y el modelado.


In [ ]:
from IPython.display import display
import pandas as pd
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer

# elegimos una reseña para realizar la comparación
index = 0  # cambiando el índice se puede trabajar con otras reseñas
texto = df.loc[index, 'texto_limpio']

# tokenización simple
tokens_simple_compara = texto.split()

# tokenización con NLTK (TweetTokenizer)
tknzr = TweetTokenizer()
tokens_nltk_compara = tknzr.tokenize(texto)

# tokenización con scikit-learn
vectorizador = CountVectorizer()
vectorizador.fit([texto])
tokens_sklearn_compara = vectorizador.get_feature_names_out()

# tabla comparativa
comparacion = pd.DataFrame({
    'Tokenización Simple': [tokens_simple_compara],
    'Tokenización NLTK (TweetTokenizer)': [tokens_nltk_compara],
    'Tokenización CountVectorizer': [list(tokens_sklearn_compara)]
})

comparacion = comparacion.T.rename(columns={0: 'Tokens'})
display(comparacion)

,Tokens
Tokenización Simple,"[uno, de, los, otros, criticos, ha, mencionado..."
Tokenización NLTK (TweetTokenizer),"[uno, de, los, otros, criticos, ha, mencionado..."
Tokenización CountVectorizer,"[acostumbre, acuerdos, adentro, agenda, al, al..."


### Conclusión: Diferencias entre Métodos de Tokenización

Al comparar los tres métodos de tokenización aplicados sobre una misma reseña, se observan las siguientes diferencias clave:

- **Tokenización Simple:** simplemente divide el texto por espacios, lo que genera errores comunes como mantener palabras unidas a signos de puntuación (por ejemplo, `"película."` en lugar de `"película"` y `"."` separados). No distingue entre signos ni realiza ningún análisis lingüístico.

- **Tokenización con NLTK (`TweetTokenizer`):** logra una separación más precisa de palabras y signos de puntuación, identificando correctamente los tokens y aislando los símbolos como unidades independientes (por ejemplo, `"."`, `"¿"`, `"¡"`). Es más robusta para textos en español y en contextos informales.

- **Tokenización con `CountVectorizer`:** extrae un vocabulario único ordenado alfabéticamente y sin duplicados. No conserva el orden original del texto ni muestra signos de puntuación como tokens válidos. Su objetivo es convertir el texto en una representación numérica, por lo que omite elementos considerados poco informativos por defecto.

En resumen, la elección del método de tokenización influye directamente en cómo se interpretan y procesan los textos. Métodos más simples pueden ser suficientes para tareas básicas, pero para análisis más detallados o sensibles a los signos, se recomienda usar tokenizadores especializados como los de NLTK.


### Conteo de Palabras

El conteo de palabras es una técnica fundamental en el análisis de texto, ya que permite identificar las palabras más frecuentes y su posible relevancia semántica dentro del corpus. En esta sección se aplicarán tres métodos distintos para contar la frecuencia de las palabras obtenidas en los procesos de tokenización:

1. **Diccionario en Python:** se construye manualmente un diccionario donde las claves son palabras y los valores son sus respectivas frecuencias.
2. **`collections.Counter`:** una herramienta especializada que simplifica y optimiza el conteo de elementos en una lista.
3. **`CountVectorizer` (de scikit-learn):** realiza automáticamente el conteo de palabras durante la vectorización del texto, generando una matriz documento-término y un vocabulario asociado.

Estos enfoques permiten comparar cómo varían los resultados en cuanto al formato, precisión y facilidad de implementación, y constituyen la base para tareas posteriores como clasificación de texto o análisis de sentimiento.

In [ ]:
# Diccionario de conteo
word_counts_dict = {}

# Verificamos el contenido de las filas antes de contar
print("Verificando filas no conformes:")
for i, tokens in enumerate(df['tokens_simple']):
    if not isinstance(tokens, list):
        print(f"Fila {i} no es una lista: {tokens}")

# Iteramos sobre cada lista de tokens en la columna
for tokens in df['tokens_simple']:
    if isinstance(tokens, list):
        for token in tokens:
            word_counts_dict[token] = word_counts_dict.get(token, 0) + 1

# Tamaño del diccionario
print(f"\nTotal de palabras únicas: {len(word_counts_dict)}")

# Ordenamos el diccionario por frecuencia
sorted_word_counts = dict(sorted(word_counts_dict.items(), key=lambda item: item[1], reverse=True))

# Mostrar las 20 palabras más frecuentes
print("\nTop 20 palabras más frecuentes:")
for palabra, frecuencia in list(sorted_word_counts.items())[:20]:
    print(f"{palabra}: {frecuencia}")

# Creamos el txt con el resultado completo
with open("conteo_dic_completo.txt", "w", encoding="utf-8") as file:
    for palabra, frecuencia in sorted_word_counts.items():
        file.write(f"{palabra}: {frecuencia}\n")

print("\nConteo completo guardado en 'conteo_dic_completo.txt'.")

Verificando filas no conformes:

Total de palabras únicas: 291893

Top 20 palabras más frecuentes:
de: 660272
la: 395037
que: 393862
y: 297323
en: 272775
el: 245835
a: 215976
un: 184852
es: 178347
una: 168990
los: 149396
pelicula: 144946
no: 138263
se: 131987
esta: 115033
para: 96414
lo: 92870
con: 90901
las: 90118
por: 88990

Conteo completo guardado en 'conteo_dic_completo.txt'.


In [ ]:
from collections import Counter

# Aplanar los tokens de las reseñas
tokens_simple_counter = [token for lista in df['tokens_simple'] if isinstance(lista, list) for token in lista]

# Usar Counter para contar frecuencias
word_counts_counter = Counter(tokens_simple_counter)

# Mostrar las 20 palabras más frecuentes
print("Conteo con Counter (ordenado por frecuencia):")
for palabra, frecuencia in word_counts_counter.most_common(20):
    print(f"{palabra}: {frecuencia}")

# Creamos el txt con el resultado completo
with open("conteo_counter_completo.txt", "w", encoding="utf-8") as f:
    for palabra, frecuencia in word_counts_counter.most_common():
        f.write(f"{palabra}: {frecuencia}\n")

print("\nConteo guardado en 'conteo_counter_completo.txt'")

Conteo con Counter (ordenado por frecuencia):
de: 660272
la: 395037
que: 393862
y: 297323
en: 272775
el: 245835
a: 215976
un: 184852
es: 178347
una: 168990
los: 149396
pelicula: 144946
no: 138263
se: 131987
esta: 115033
para: 96414
lo: 92870
con: 90901
las: 90118
por: 88990

Conteo guardado en 'conteo_counter_completo.txt'


In [ ]:
# Conteo de palabras con CountVectorizer sin densificar
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

# Aplicar CountVectorizer
corpus = df['texto_limpio'].tolist()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

# Obtener las palabras y sus frecuencias totales sin densificar
frecuencias = X.sum(axis=0).A1  # A1 vuelve el resultado en un array 1D

# Crear el diccionario palabra-frecuencia
conteo_vect = dict(zip(vectorizer.get_feature_names_out(), frecuencias))

# Ordenar por frecuencia descendente
conteo_vect_ordenado = dict(sorted(conteo_vect.items(), key=lambda x: x[1], reverse=True))

#Mostrar las 20 palabras más frecuentes
print("Top 20 palabras más frecuentes (CountVectorizer - corpus completo):")
for palabra, frecuencia in list(conteo_vect_ordenado.items())[:20]:
    print(f"{palabra}: {frecuencia}")

# Grabar el resultado en un txt
with open("conteo_countVectorizer_completo.txt", "w", encoding='utf-8') as f:
    for palabra, frecuencia in conteo_vect_ordenado.items():
        f.write(f"{palabra}: {frecuencia}\n")

print("\nConteo guardado en 'conteo_countVectorizer_completo.txt'")


Top 20 palabras más frecuentes (CountVectorizer - corpus completo):
de: 660272
la: 395037
que: 393862
en: 272775
el: 245835
un: 184852
es: 178347
una: 168990
los: 149396
pelicula: 144946
no: 138263
se: 131987
esta: 115033
para: 96414
lo: 92870
con: 90901
las: 90118
por: 88990
su: 87598
como: 83458

Conteo guardado en 'conteo_countVectorizer_completo.txt'


### Comparación de las Palabras Más Frecuentes según el Método de Tokenización

A continuación se presenta una tabla que muestra las 20 palabras más frecuentes extraídas del conjunto de reseñas mediante tres métodos distintos de tokenización:

- **Tokenización Simple**: utiliza el método `.split()` sobre cadenas de texto.
- **Tokenización NLTK**: se implementa con `TweetTokenizer`, el cual maneja mejor signos de puntuación y emoticones, útil para texto informal.
- **Tokenización scikit-learn**: se realiza mediante `CountVectorizer`, que incluye su propio sistema de limpieza y tokenización interna.

Esto permite observar cómo varía la frecuencia de palabras comunes dependiendo del método utilizado.

Las palabras más frecuentes son generalmente artículos, preposiciones y conjunciones, como "la", "de", "y", que suelen ser palabras de parada. Aunque los tres métodos coinciden en gran medida, pueden diferir en cómo tratan los signos de puntuación o los tokens compuestos.

In [ ]:
# Tabla comparativa de las N palabras más frecuentes por método
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
import pandas as pd

# tokenización simple (split)
tokens_split = []
for texto in reseñas:
    tokens_split.extend(texto.lower().split())
frecuencia_split = [pal for pal, _ in Counter(tokens_split).most_common(20)]

# tokenización con NLTK (TweetTokenizer)
tokenizer = TweetTokenizer()
tokens_nltk = []
for texto in reseñas:
    tokens_nltk.extend(tokenizer.tokenize(texto.lower()))
frecuencia_nltk = [pal for pal, _ in Counter(tokens_nltk).most_common(20)]

# tokenización con CountVectorizer
vectorizer = CountVectorizer()
X_vect = vectorizer.fit_transform(reseñas)
vocab = vectorizer.get_feature_names_out()
frecuencias_vect = X_vect.sum(axis=0).A1
top_vect = sorted(zip(vocab, frecuencias_vect), key=lambda x: -x[1])[:20]
frecuencia_vect = [pal for pal, _ in top_vect]

# creamos la tabla comparativa
df_tokens = pd.DataFrame({
    'Tokenización Simple': frecuencia_split,
    'Tokenización NLTK': frecuencia_nltk,
    'Tokenización scikit-learn': frecuencia_vect
})
df_tokens.insert(0, 'Rank', range(1, len(df_tokens) + 1))
df_tokens

,Rank,Tokenización Simple,Tokenización NLTK,Tokenización scikit-learn
0,1,de,de,de
1,2,la,la,la
2,3,que,que,que
3,4,y,y,en
4,5,en,en,el
5,6,el,el,un
6,7,a,a,es
7,8,un,un,una
8,9,es,es,los
9,10,una,una,pelicula


## Paso 5: Creación de la Bolsa de Palabras (Bag of Words - BoW)

La Bolsa de Palabras (BoW) es una técnica fundamental en el procesamiento de texto que convierte un conjunto de documentos en una representación numérica basada en la frecuencia de las palabras.

Este modelo:
- Ignora el orden de las palabras.
- Representa cada documento como un vector.
- Usa como base el vocabulario total encontrado en el corpus.

Usaremos la clase `CountVectorizer` de `scikit-learn` para crear esta representación a partir de las reseñas en español. Esta herramienta se encarga tanto de la tokenización como de construir la matriz documento-término.

Los pasos son:
1. Inicializar el vectorizador.
2. Ajustarlo a los datos (fit).
3. Transformar el texto en vectores.
4. Inspeccionar la matriz y el vocabulario generado.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Inicializamos el vectorizador
vectorizer = CountVectorizer()

# Aplicamos CountVectorizer
X_bow = vectorizer.fit_transform(reseñas)

# Mostramos la forma de la matriz (documentos x términos)
print(f"Matriz BoW: {X_bow.shape[0]} documentos, {X_bow.shape[1]} términos únicos")

# Obtenemos el vocabulario (palabras e índices)
vocabulario = vectorizer.vocabulary_
print("Ejemplo del vocabulario generado (10 primeras palabras):")
print(dict(list(vocabulario.items())[:10]))

# Obtenemos los nombre de las características (palabras)
caracteristicas = vectorizer.get_feature_names_out()

# Si deseas inspeccionar, podemos densificar solamente las 5 primeros documentos
df_bow = pd.DataFrame(
    X_bow[:5].toarray(),
    columns=caracteristicas
)

print("Primeras filas de la matriz BoW:")
print(df_bow)


Matriz BoW: 50000 documentos, 291854 términos únicos
Ejemplo del vocabulario generado (10 primeras palabras):
{'uno': 277067, 'de': 69676, 'los': 165036, 'otros': 197030, 'criticos': 66027, 'ha': 128069, 'mencionado': 174917, 'que': 220898, 'despues': 77294, 'ver': 280412}
Primeras filas de la matriz BoW:
   00  000  0000000000001  00000001  000001  0001  00015  001  002  00383042  \
0   0    0              0         0       0     0      0    0    0         0   
1   0    0              0         0       0     0      0    0    0         0   
2   0    0              0         0       0     0      0    0    0         0   
3   0    0              0         0       0     0      0    0    0         0   
4   0    0              0         0       0     0      0    0    0         0   

   ...  zzzz  zzzzip  zzzzz  zzzzzse  zzzzzzzzz  zzzzzzzzzzzzzz  \
0  ...     0       0      0        0          0               0   
1  ...     0       0      0        0          0               0   
2  ...     

### Representación BoW con N-gramas

Hasta ahora, la representación de Bolsa de Palabras se generó utilizando unigramas, es decir, palabras individuales. Sin embargo, `CountVectorizer` permite incluir también n-gramas, como bigramas (pares de palabras consecutivas), lo cual enriquece la representación al capturar un poco más de contexto.

Por ejemplo, expresiones como "muy buena" o "no me gustó" pueden ser representadas como unidades, lo que es útil para modelos que intentan capturar patrones de opinión o sentimiento.

A continuación, se genera una nueva matriz BoW utilizando `ngram_range=(1, 2)` para incluir tanto unigramas como bigramas.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# inicializamos CountVectorizer con n-gramas (unigramas + bigramas)
vectorizer_ngram = CountVectorizer(ngram_range=(1, 2))

X_bow_ngram = vectorizer_ngram.fit_transform(reseñas)

# información general de la matriz
print(f"Matriz BoW con unigramas y bigramas:")
print(f"  - Documentos: {X_bow_ngram.shape[0]}")
print(f"  - Términos únicos: {X_bow_ngram.shape[1]}")

# mostramos las primeras 20 palabras
vocabulario_ngram = vectorizer_ngram.get_feature_names_out()
print("\nEjemplo de 20 términos del vocabulario (unigramas y bigramas):")
print(vocabulario_ngram[:20])

Matriz BoW con unigramas y bigramas:
  - Documentos: 50000
  - Términos únicos: 3025428

Ejemplo de 20 términos del vocabulario (unigramas y bigramas):
['00' '00 10' '00 agent' '00 agente' '00 doc' '00 en' '00 horas'
 '00 incluida' '00 incluso' '00 las' '00 para' '00 pero' '00 regresa'
 '00 schneider' '00 se' '00 vale' '00 ya' '000' '000 000' '000 para']


#Análisis de sentimiento con Regresión Logística

## 3. Entrenamiento Base del Modelo
Se entrena el modelo Multinomial Naive Bayes utilizando el texto limpio.

## Implementación del Modelo MultinomialNB (baseline)

In [ ]:
# Carga de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Javier\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
# Vectorización y Entrenamiento (baseline):
  # Trabajaremos con Bag of Words con `CountVectorizer`
  # Luego se divide el dataset en entrenamiento y prueba (80/20)
  # Y se entrena un clasificador MultinomialNB

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['texto_limpio'])
y = df['sentimiento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo_baseline = MultinomialNB()
modelo_baseline.fit(X_train, y_train)
y_pred = modelo_baseline.predict(X_test)

# Evaluación
print("Informe de clasificación - Baseline:")
print(classification_report(y_test, y_pred, digits=4))


Informe de clasificación - Baseline:
              precision    recall  f1-score   support

    negativo     0.8038    0.8772    0.8389      4961
    positivo     0.8672    0.7892    0.8264      5039

    accuracy                         0.8329     10000
   macro avg     0.8355    0.8332    0.8327     10000
weighted avg     0.8358    0.8329    0.8326     10000



In [ ]:
# Métricas del Corpus (baseline): Se calculan las métricas del corpus para evaluar la riqueza léxica y redundancia:
  # - Total de tokens únicos
  # - Tokens promedio por documento
  # - Tamaño del vocabulario BoW

tokens = [palabra for texto in df['texto_limpio'] for palabra in texto.split()]
tokens_unicos = set(tokens)
tokens_total = len(tokens)
tokens_promedio_doc = tokens_total / len(df)
vocabulario_bow = len(vectorizer.get_feature_names_out())

print(f"Tokens únicos: {len(tokens_unicos)}")
print(f"Tokens promedio por documento: {tokens_promedio_doc:.2f}")
print(f"Tamaño del vocabulario BoW: {vocabulario_bow}")


Tokens únicos: 291893
Tokens promedio por documento: 236.00
Tamaño del vocabulario BoW: 291854


# Técnicas de reducción y normalización

# Stemming

Aplicamos el algoritmo SnowballStemmer en español para reducir las palabras a su raíz. Luego, vectorizamos nuevamente, entrenamos el modelo y comparamos métricas con el baseline.

In [ ]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('spanish')

def aplicar_stemming(texto):
    palabras = texto.split()
    palabras_stem = [stemmer.stem(palabra) for palabra in palabras]
    return ' '.join(palabras_stem)

df['texto_stem'] = df['texto_limpio'].apply(aplicar_stemming)
df[['texto_limpio', 'texto_stem']].head()

,texto_limpio,texto_stem
0,uno de los otros criticos ha mencionado que de...,uno de los otros critic ha mencion que despu d...
1,una pequena pequena produccionla tecnica de fi...,una pequen pequen produccionl tecnic de filmac...
2,pense que esta era una manera maravillosa de p...,pens que esta era una maner maravill de pas ti...
3,basicamente hay una familia donde un nino pequ...,basic hay una famili dond un nin pequen jak pi...
4,el amor en el tiempo de petter mattei es una p...,el amor en el tiemp de pett mattei es una peli...


In [ ]:
# A partir de lo obtenido después de aplicar SnowballStemmer:
  # Vectorizamos nuevamente
  # Entrenamos el modelo
  # Y comparamos métricas con el baseline.

vectorizer_stem = CountVectorizer()
X_stem = vectorizer_stem.fit_transform(df['texto_stem'])
y = df['sentimiento']

X_train_stem, X_test_stem, y_train_stem, y_test_stem = train_test_split(X_stem, y, test_size=0.2, random_state=42)

modelo_stem = MultinomialNB()
modelo_stem.fit(X_train_stem, y_train_stem)
y_pred_stem = modelo_stem.predict(X_test_stem)

print("Informe de clasificación - Stemming:")
print(classification_report(y_test_stem, y_pred_stem, digits=4))

Informe de clasificación - Stemming:
              precision    recall  f1-score   support

    negativo     0.7925    0.8720    0.8303      4961
    positivo     0.8602    0.7752    0.8154      5039

    accuracy                         0.8232     10000
   macro avg     0.8263    0.8236    0.8229     10000
weighted avg     0.8266    0.8232    0.8228     10000



In [ ]:
tokens_stem = [palabra for texto in df['texto_stem'] for palabra in texto.split()]
tokens_unicos_stem = set(tokens_stem)
tokens_total_stem = len(tokens_stem)
tokens_promedio_doc_stem = tokens_total_stem / len(df)
vocabulario_bow_stem = len(vectorizer_stem.get_feature_names_out())

print(f"Tokens únicos: {len(tokens_unicos_stem)}")
print(f"Tokens promedio por documento: {tokens_promedio_doc_stem:.2f}")
print(f"Tamaño del vocabulario BoW: {vocabulario_bow_stem}")

Tokens únicos: 218304
Tokens promedio por documento: 236.00
Tamaño del vocabulario BoW: 218265


# Lematización:
Se utiliza la biblioteca spaCy con el modelo en español `es_core_news_sm` para extraer los lemas de cada palabra en las reseñas.
Luego se entrena nuevamente el modelo y se miden los resultados.

In [ ]:
from tqdm import tqdm
import spacy
import pandas as pd

# Carga el modelo de SpaCy para español
nlp = spacy.load("es_core_news_sm")

# Definimos la función de lematización
def lematizar(texto):
    doc = nlp(texto)
    lemas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
    return ' '.join(lemas)


# Lista para almacenar resultados
lematizados = []

# Aplicar lematización con barra de progreso
for texto in tqdm(df['texto_limpio'], desc="Lematizando"):
    lematizados.append(lematizar(texto))

# Guardar resultados en nueva columna
df['texto_lemma'] = lematizados

# Mostrar primeras filas para verificar
print(df[['texto_limpio', 'texto_lemma']].head())



























































































































Exception ignored in: <function tqdm.__del__ at 0x000002A3903F2F20>
Traceback (most recent call last):
  File "c:\Users\Javier\.virtualenvs\LittleLemon-zoWj-mv7\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\Javier\.virtualenvs\LittleLemon-zoWj-mv7\Lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'





















































































































































































































































































































































































































                                        texto_limpio  \
0  uno de los otros criticos ha mencionado que de...   
1  una pequena pequena produccionla tecnica de fi...   
2  pense que esta era una manera maravillosa de p...   
3  basicamente hay una familia donde un nino pequ...   
4  el amor en el tiempo de petter mattei es una p...   

                                         texto_lemma  
0  uno de el otro critico haber mencionar que des...  
1  uno pequén pequén produccionla tecnico de film...  
2  pensar que este ser uno manera maravilloso de ...  
3  basicamente haber uno familia donde uno nino p...  
4  el amor en el tiempo de petter mattei ser uno ...  


In [ ]:
# Entrenamos nuevamente el modelo
vectorizer_lemma = CountVectorizer()
X_lemma = vectorizer_lemma.fit_transform(df['texto_lemma'])

X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma = train_test_split(X_lemma, y, test_size=0.2, random_state=42)

modelo_lemma = MultinomialNB()
modelo_lemma.fit(X_train_lemma, y_train_lemma)
y_pred_lemma = modelo_lemma.predict(X_test_lemma)

print("Informe de clasificación - Lematización:")
print(classification_report(y_test_lemma, y_pred_lemma, digits=4))

Informe de clasificación - Lematización:
              precision    recall  f1-score   support

    negativo     0.7946    0.8772    0.8339      4961
    positivo     0.8654    0.7767    0.8187      5039

    accuracy                         0.8266     10000
   macro avg     0.8300    0.8270    0.8263     10000
weighted avg     0.8303    0.8266    0.8262     10000



In [ ]:
tokens_lemma = [palabra for texto in df['texto_lemma'] for palabra in texto.split()]
tokens_unicos_lemma = set(tokens_lemma)
tokens_total_lemma = len(tokens_lemma)
tokens_promedio_doc_lemma = tokens_total_lemma / len(df)
vocabulario_bow_lemma = len(vectorizer_lemma.get_feature_names_out())

print(f"Tokens únicos: {len(tokens_unicos_lemma)}")
print(f"Tokens promedio por documento: {tokens_promedio_doc_lemma:.2f}")
print(f"Tamaño del vocabulario BoW: {vocabulario_bow_lemma}")

Tokens únicos: 275877
Tokens promedio por documento: 237.09
Tamaño del vocabulario BoW: 275839


# Eliminación de StopWords:
Aplicamos la eliminación de stopwords utilizando CountVectorizer con una lista de stopwords en español obtenida desde `nltk` para eliminar las StopWords en español.
Luego, se entrena el modelo y se evalúa el desempeño.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Obtenemos la lista de stopwords en español
stopwords_es = stopwords.words('spanish')

# Usamos CountVectorizer con la lista personalizada de stopwords
vectorizer_stopwords = CountVectorizer(stop_words=stopwords_es)

X_stopwords = vectorizer_stopwords.fit_transform(df['texto_limpio'])

X_train_stopwords, X_test_stopwords, y_train_stopwords, y_test_stopwords = train_test_split(X_stopwords, y, test_size=0.2, random_state=42)

modelo_stopwords = MultinomialNB()
modelo_stopwords.fit(X_train_stopwords, y_train_stopwords)
y_pred_stopwords = modelo_stopwords.predict(X_test_stopwords)

print("Informe de clasificación - StopWords:")
print(classification_report(y_test_stopwords, y_pred_stopwords, digits=4))

Informe de clasificación - StopWords:
              precision    recall  f1-score   support

    negativo     0.8157    0.8760    0.8448      4961
    positivo     0.8684    0.8051    0.8355      5039

    accuracy                         0.8403     10000
   macro avg     0.8420    0.8406    0.8402     10000
weighted avg     0.8422    0.8403    0.8401     10000



In [ ]:
# Convertimos stopwords a un conjunto para mejorar el rendimiento (porque tardaba mucho con la lista)
stopwords_es_set = set(stopwords.words('spanish'))

# Obtener tokens únicos sin las stopwords
tokens_stopwords = [palabra for texto in df['texto_limpio'] for palabra in texto.split() if palabra not in stopwords_es_set]
tokens_unicos_stopwords = set(tokens_stopwords)
tokens_total_stopwords = len(tokens_stopwords)
tokens_promedio_doc_stopwords = tokens_total_stopwords / len(df)
vocabulario_bow_stopwords = len(vectorizer_stopwords.get_feature_names_out())

print(f"Tokens únicos: {len(tokens_unicos_stopwords)}")
print(f"Tokens promedio por documento: {tokens_promedio_doc_stopwords:.2f}")
print(f"Tamaño del vocabulario BoW: {vocabulario_bow_stopwords}")

Tokens únicos: 291707
Tokens promedio por documento: 129.82
Tamaño del vocabulario BoW: 291672


## Análisis Comparativo

# Rendimiento del modelo

En esta sección se presentan los resultados de evaluación del clasificador **Multinomial Naive Bayes** bajo distintos esquemas de preprocesamiento textual. Se muestran las métricas de **accuracy**, **precisión**, **recall** y **F1-score**, expresadas en porcentaje (%), para cada variante aplicada.

El objetivo es observar cómo afectan al rendimiento del modelo técnicas como **Stemming**, **Lematización** y **eliminación de StopWords**, tanto por separado como combinadas.

A continuación, se presenta una tabla comparativa con los resultados:

> **Tabla 1. Rendimiento del modelo MultinomialNB según técnica de preprocesamiento**

In [ ]:
from sklearn.metrics import classification_report

def obtener_metricas(y_true, y_pred):
    report = classification_report(y_true, y_pred, output_dict=True)
    labels = [k for k in report.keys() if k not in ("accuracy", "macro avg", "weighted avg")]
    precision = sum(report[label]["precision"] for label in labels) / len(labels)
    recall = sum(report[label]["recall"] for label in labels) / len(labels)
    f1 = sum(report[label]["f1-score"] for label in labels) / len(labels)
    accuracy = report["accuracy"]
    # Convertir a porcentaje
    return [round(accuracy * 100, 2), round(precision * 100, 2), round(recall * 100, 2), round(f1 * 100, 2)]

# Cargamos resultados
resultados_obtenidos = {
    "Sin normalización (baseline)": obtener_metricas(y_test, y_pred),
    "Stemming": obtener_metricas(y_test_stem, y_pred_stem),
    "Lematización": obtener_metricas(y_test_lemma, y_pred_lemma),
    "StopWords": obtener_metricas(y_test_stopwords, y_pred_stopwords),
}

# Creamos dataFrame y mostramos
df_resultados = pd.DataFrame.from_dict(
    resultados_obtenidos,
    orient='index',
    columns=["Accuracy (%)", "Precisión (%)", "Recall (%)", "F1-score (%)"]
)

df_resultados = df_resultados.sort_values("Accuracy (%)", ascending=False)
display(df_resultados)

,Accuracy (%),Precisión (%),Recall (%),F1-score (%)
StopWords,84.03,84.20,84.06,84.02
Sin normalización (baseline),83.29,83.55,83.32,83.27
Lematización,82.66,83.00,82.70,82.63
Stemming,82.32,82.63,82.36,82.29


# Análisis de reducción del corpus

Además del rendimiento del modelo, se evalúa cómo cada técnica de preprocesamiento impacta en la **reducción y simplificación del corpus textual**. Este análisis se enfoca en tres aspectos clave:

- **Tokens únicos**: cantidad total de palabras distintas luego del preprocesamiento.
- **Tokens promedio por documento**: medida de la longitud media de los textos.
- **Tamaño del vocabulario BoW**: cantidad de palabras distintas utilizadas por `CountVectorizer` al construir la matriz de características.

A continuación se resume esta información en una tabla:

> **Tabla 2. Reducción del corpus según técnica de preprocesamiento**

In [ ]:
# Recolecctamos estadísticas de corpus
corpus_stats = {
    "Sin normalización (baseline)": [
        len(set([palabra for texto in df["texto_limpio"] for palabra in texto.split()])),
        len([palabra for texto in df["texto_limpio"] for palabra in texto.split()]) / len(df),
        len(vectorizer.get_feature_names_out())
    ],
    "Stemming": [
        len(set([palabra for texto in df["texto_stem"] for palabra in texto.split()])),
        len([palabra for texto in df["texto_stem"] for palabra in texto.split()]) / len(df),
        len(vectorizer_stem.get_feature_names_out())
    ],
    "Lematización": [
        len(set([palabra for texto in df["texto_lemma"] for palabra in texto.split()])),
        len([palabra for texto in df["texto_lemma"] for palabra in texto.split()]) / len(df),
        len(vectorizer_lemma.get_feature_names_out())
    ],
    "StopWords": [
        len(set([palabra for texto in df["texto_limpio"] for palabra in texto.split() if palabra not in stopwords_es_set])),
        len([palabra for texto in df["texto_limpio"] for palabra in texto.split() if palabra not in stopwords_es_set]) / len(df),
        len(vectorizer_stopwords.get_feature_names_out())
    ],
}

# Creamos la tabla
df_corpus = pd.DataFrame.from_dict(
    corpus_stats,
    orient="index",
    columns=["Tokens únicos", "Tokens promedio/doc", "Vocabulario BoW"]
)

# Redondeamos valores para una mejor visualización
df_corpus["Tokens promedio/doc"] = df_corpus["Tokens promedio/doc"].round(2)

display(df_corpus)

,Tokens únicos,Tokens promedio/doc,Vocabulario BoW
Sin normalización (baseline),291893,236.00,291854
Stemming,218304,236.00,218265
Lematización,275877,237.09,275839
StopWords,291707,129.82,291672


## Conclusiones

A partir de los resultados obtenidos, es posible extraer las siguientes conclusiones respecto al impacto de las técnicas de preprocesamiento textual en el rendimiento y la complejidad del modelo MultinomialNB:

- **Eliminación de StopWords fue la técnica que logró el mejor rendimiento general**, alcanzando un **accuracy del 83.79%**, superior incluso al baseline. Esto sugiere que eliminar palabras vacías contribuye a mejorar la discriminación entre clases, probablemente al reducir ruido y enfocarse en términos relevantes para la clasificación.

- El **modelo baseline (sin normalización)** obtuvo un rendimiento competitivo (83.29%) sin aplicar técnicas adicionales. Sin embargo, presentó el vocabulario más extenso y una gran cantidad de tokens únicos, lo que puede implicar mayor costo computacional.

- **Lematización** y **Stemming** no mejoraron el rendimiento del modelo; de hecho, produjeron una leve disminución de métricas, especialmente en el caso del Stemming. Aunque estas técnicas reducen el tamaño del vocabulario (en especial el stemming, que bajó de ~290K a ~210K palabras únicas), esta simplificación no se tradujo en una mejora del desempeño. Esto sugiere que una reducción excesiva puede perder información semántica útil para la clasificación.

- En cuanto a la **reducción del corpus**, la eliminación de stopwords fue la técnica que más disminuyó la longitud promedio de los documentos (de ~235 a ~126 tokens/doc), pero no redujo significativamente el vocabulario total. Por el contrario, **Stemming** fue el que más redujo el vocabulario y la diversidad léxica, aunque a costa de rendimiento.

- En términos de eficiencia y equilibrio, **la eliminación de StopWords se presenta como la técnica más efectiva**, ya que logra una mejora en las métricas del modelo con una reducción considerable en la longitud de los textos, sin comprometer la riqueza léxica de manera significativa.

En síntesis, las decisiones de preprocesamiento deben balancear la simplificación del corpus con la preservación de la información. En este caso, eliminar stopwords fue la estrategia más beneficiosa tanto en rendimiento como en eficiencia.